In [7]:
from dotenv import load_dotenv
from anthropic import Anthropic
import json

load_dotenv()
client = Anthropic()
model = "claude-3-haiku-20240307"

In [2]:
def add_user_message(messages, text):
    messages.append({"role": "user", "content": text})
    return messages

def add_assistent_messages(messages, text):
    messages.append({"role": "assistant", "content": text})
    return messages

def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
            "model": model,
            "max_tokens": 1000,
            "messages": messages,
            "temperature":temperature,
            "stop_sequences":stop_sequences,
    }
    if system:
        params["system"] = system

    message = client.messages.create(
        **params
    )
    return message.content[0].text


In [8]:
def generate_dataset():
    prompt = """
Generate an evaluation dataset for a prompt evaluation. The dataset will be used to evaluate prompts that generate Python, JSON, or Regex specifically for AWS-related tasks. Generate an array of JSON objects, each representing task that requires Python, JSON, or a Regex to complete.

Example output:
```json
[
  {
    "task": "Description of task",
  },
  ...additional
]
```

* Focus on tasks that can be solved by writing a single Python function, a single JSON object, or a single regex
* Focus on tasks that do not require writing much code

Please generate 3 objects.
"""
    messages = []

    add_user_message(messages, prompt)
    add_assistent_messages(messages, "```json")
    answer = chat(messages, stop_sequences=["```"])
    return json.loads(answer)

In [11]:
test_dataset = generate_dataset()
test_dataset

[{'task': 'Write a Python function to create an AWS S3 bucket with a given name.'},
 {'task': 'Create a JSON object to configure an AWS Lambda function with a specified runtime, memory size, and timeout.'},
 {'task': "Write a regular expression to validate an AWS EC2 instance ID in the format 'i-0123456789abcdef'."}]

In [13]:
with open("data/dataset.json", "w") as f:
    json.dump(test_dataset, f, indent=2)